In [8]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
tf.config.experimental.list_physical_devices('GPU')
import datetime
from keras.models import Sequential
from keras.layers import Conv2D, Dense, AveragePooling2D, Dropout, Flatten

<h1>Extract MNIST data</h1>

In [ ]:
#get mnist data, with one_hot encoding
mnist = tf.keras.datasets.mnist.load_data(path="mnist.npz")
(train_images, train_labels), (test_images, test_labels) = mnist
tl = train_labels
train_images = train_images.astype("float32")/255.0
test_images = test_images.astype("float32")/255.0
train_labels = tf.one_hot(train_labels,depth=10)
test_labels = tf.one_hot(test_labels, depth=10)

<h1>Prepare training, validation and testing data</h1>

In [ ]:
#pad images with 0s (28x28 to 32x32)
x_train = np.pad(train_images, ((0,0),(2,2),(2,2)), 'constant')
x_test = np.pad(test_images, ((0,0),(2,2),(2,2)), 'constant')
def plot_digits(X, Y):
    for i in range(20):
        plt.subplot(5, 4, i+1)
        plt.tight_layout()
        plt.imshow(X[i].reshape(32, 32), cmap='gray')
        plt.title('Digit:{}'.format(Y[i]))
        plt.xticks([])
        plt.yticks([])
    plt.show()
plot_digits(x_train, tl)
x_train = x_train.reshape(x_train.shape[0], 32, 32, 1)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 1)
print(x_train.shape)

In [ ]:
print(x_train.shape, train_labels.shape)
print(x_test.shape, test_labels.shape)

<h1>Define LeNet-5</h1>

In [ ]:
"""
Define Lenet-5 model
"""
model = Sequential()
# convolution 1
model.add(Conv2D(filters=6, kernel_size=(5,5), strides=1, activation="tanh", input_shape=(32, 32, 1)))
# pooling 1 (subsampling)
model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# convolution 2
model.add(Conv2D(filters=16, kernel_size=(5,5), strides=1, activation="tanh", padding="valid"))
# pooling 2 (subsampling)
model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# flatten the convolution to connect with full connectted layer
model.add(Flatten())
# Fully connected layer
model.add(Dense(84, activation="tanh"))
# output layer
model.add(Dense(10, activation="softmax"))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
)

model.summary()




<h1>Training, validating, testing</h1>
<h2>1. Print out validation accuracy after each training epoch</h2>
<h2>2. Print out training time on each epoch</h2>
<h2>3. Print out testing accuracy</h2>

In [ ]:
history = model.fit(x_train, train_labels, epochs=15, validation_data=(x_test, test_labels))

In [ ]:
model.evaluate(x_test, test_labels)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])
axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])